In [ ]:
import cv2
from ultralytic import YOLO
import os

ModuleNotFoundError: No module named 'ultralytic'

In [ ]:
def process_frames_folder(folder_path):
    print(f"Loading the Model...")

    model = YOLO('yolov8n.pt')

    image_patterns = os.path.
